# Transportation Problem: MODI

## 1. Balance supply and demand
In case the demand is more than supply, then dummy origin is added to the table. The supply of dummy origin will be equal to the difference between the total supply and total demand. The cost associated with the dummy origin will be zero (or other supply/demand-related cost)

In [1]:
import numpy as np

def balance_supply_demand(C, supply, demand):
    C = C.copy()
    supply = supply.copy()
    demand = demand.copy()
    s = sum(supply)
    d = sum(demand)
    
    if s < d:
        # Create artificial supply row
        supply.append(d - s)
        C = np.append(C, [C[-1, :]*0], axis=0)
    elif s > d:
        demand.append(s - d)
        # Create artificial demand column
        C = np.append(C, np.zeros((C.shape[0], 1)), axis=1)
    return C, supply, demand

## 2. Obtain the initial feasable solution

Identifying the solution that satisfies the requirements of demand and supply. 

### Northwest Corner Rule

The North-West Corner Rule is a method adopted to compute the initial feasible solution of the transportation problem. The name North-west corner is given to this method because the basic variables are selected from the extreme left corner. Doesn't consider transportation costs.

<img src="https://www.linearprogramming.info/wp-content/uploads/2015/10/Northwest-Corner-Model-solu.gif"/>

In [2]:
def northwest_corner_rule(C, supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    Q = np.empty(C.shape)
    Q[:] = np.nan
    i, j = 0, 0
    
    while sum(supply) > 0 and sum(demand) > 0:
        min_q = min(supply[i], demand[j])
        Q[i, j] = min_q
        supply[i] -= min_q
        demand[j] -= min_q
        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1
    return Q

### Least Cost Method

The Least Cost Method is another method used to obtain the initial feasible solution for the transportation problem. Here, the allocation begins with the cell which has the minimum cost. The lower cost cells are chosen over the higher-cost cell with the objective to have the least cost of transportation.

- In which cell we shall allocate? Generally, the cost where maximum quantity can be assigned should be chosen to obtain the better initial solution

In [3]:
def matrix_min_rule(C, supply, demand):
    C = C.copy()
    supply = supply.copy()
    demand = demand.copy()
    Q = np.empty(C.shape)
    Q[:] = np.nan
    
    while sum(supply) > 0 and sum(demand) > 0:
        i, j = np.unravel_index(C.argmin(), C.shape)
        min_q = min(supply[i], demand[j])
        if min_q != 0:
            Q[i, j] = min_q
            supply[i] -= min_q
            demand[j] -= min_q
        C[i, j] = C.max() + 1 # creates a ring of min values
    return Q

## 3. Test degeneration

- If the solution is degenerate, make use of an artificial quantity ($=0$)
- The quantity is assigned to unoccupied cell, which has the minimum transportation cost
- If quantity is inside of loop and negative (its already $0$ and cannot be reduced further), recalculate using another cell

In [4]:
def degeneration_test(C, Q, blacklist):
    C = C.copy()
    Q = Q.copy()
    s, d = Q.shape
    artvars = (s + d) - np.count_nonzero(~np.isnan(Q)) - 1
    if artvars == 0:
        return Q
    for artvar in range(artvars):
        while True:
            i, j = np.unravel_index(C.argmin(), C.shape)
            C[i, j] = C.max() + 1
            if np.isnan(Q[i, j]) and (i, j) not in blacklist:
                # Place artificial quantity (=0) to the next global min
                Q[i, j] = 0
                break
    return Q

## 4. Calculate opportunity costs
1. Calculate the values of $u_i$ and $v_j$ by using the equation $u_i+v_j = c_{ij}$
    - Introduce the variables $u_i$ (rows) and $v_i$ (columns). Set $u$ to $0$ (Convenient rule is to select the ui, which has the largest number of allocations in its row)
    - For each occupied cell calculate $u_i$ and $v_i$ using equation $c_{ij}=u_i+v_i$.
2. Calculate the opportunity costs (or reduced costs) of the unoccupied cells by using the formula $c_{ij}-u_i+v_i$ (which becomes the same value using Stepping Stone)
3. Choose the largest opportunity cost and start from there

In [5]:
def opportunity_costs(C, Q):
    # Calculate u's and v's
    QC = Q * 0 + C
    u = np.zeros(C.shape[0])
    u[1:] = np.nan
    v = np.empty(C.shape[1])
    v[:] = np.nan
    nonnans = np.argwhere(~np.isnan(Q))
    while any(np.isnan(u)) or any(np.isnan(v)):
        for i, j in nonnans:
            if ~np.isnan(u[i]) and ~np.isnan(v[j]):
                continue
            if ~np.isnan(u[i]):
                v[j] = QC[i, j] - u[i]
            elif ~np.isnan(v[j]):
                u[i] = QC[i, j] - v[j]
          
    # Calculate opportunity (reduced) costs
    OC = np.empty(C.shape)
    OC[:] = np.nan
    nans = np.argwhere(np.isnan(Q))
    for i, j in nans:
        OC[i, j] = C[i, j] - u[i] - v[j]
    return OC

## 5. Find a closed path
- Closed loops are formed starting in an empty cell
- Go to an allocated cell, then another and another until returning to empty cell

In [6]:
def find_path(Q, stop_ij, from_ij, mode):
    from_i, from_j = from_ij
    if mode == 'v':
        # Find a candidate vertically
        for i in range(Q.shape[0]):
            if i != from_i and ~np.isnan(Q[i, from_j]):
                if (i, from_j) == stop_ij:
                    # Stop iteration if initial cell found
                    return [from_ij, stop_ij]
                # Next candidate found, continue from it recursivelly
                path = find_path(Q, stop_ij, (i, from_j), 'h')
                # Return if initial cell in the path
                if path and stop_ij in path:
                    return [from_ij] + path
    elif mode == 'h':
        # Find a candidate horizontally
        for j in range(Q.shape[1]):
            if j != from_j and ~np.isnan(Q[from_i, j]):
                if (from_i, j) == stop_ij:
                    return [from_ij, stop_ij]
                path = find_path(Q, stop_ij, (from_i, j), 'v')
                if path and stop_ij in path:
                    return [from_ij] + path
                
def closed_path(Q, from_ij):
    path = find_path(Q, from_ij, from_ij, 'v')
    if path is not None:
        return path[:-1]

## 6. Reallocate quantities
- Look at the giving cells around and shift the minimum to the unallocated cell, while maintaining the feasibility constraints of supply and demand
- Other quantities must remain positive

## 7. Testing the optimality of the solution
- Repeat the process until opportunity costs are all positive.

In [66]:
def MODI(C, supply, demand):
    # Balance supply-demand-equation
    C, supply, demand = balance_supply_demand(C, supply, demand)
    
    # Make initial feasible solution
    Q = matrix_min_rule(C, supply, demand)
    
    blacklist = []
    while True:
        # Solve degeneration
        DQ = degeneration_test(C, Q, blacklist)
    
        # Calculate the opportunity costs
        OC = opportunity_costs(C, DQ)
        if np.all(OC[~np.isnan(OC)] >= 0):
            # Terminate if all opportunity costs positive
            break
        
        # Select the cell with the maximum opportunity costs
        empty_index = np.unravel_index(np.nanargmin(OC), OC.shape)
        DQ[empty_index] = 0
        
        # Find a closed path from the cell
        path = closed_path(DQ, empty_index)
        
        invalid_artval = False
        if path:
            signs = [1 if i % 2 == 0 else -1 for i in range(len(path))]
            quantities = [DQ[i, j] for i, j in path]
            
            # Check for negative zero quantities
            for index, sign, quantity in zip(path, signs, quantities):
                if sign == -1 and quantity == 0:
                    # Select another artificial quantity
                    blacklist.append(index)
                    invalid_artval = True
                    break
                    
            if invalid_artval:
                continue
            
            # Choose maximum allowed quantity to be subtracted
            maxq = min(map(lambda x: x[1], filter(lambda x: x[0] < 0, zip(signs, quantities))))
            
            # Reallocate quantities
            for index, sign, quantity in zip(path, signs, quantities):
                DQ[index] += sign * maxq
                if DQ[index] == 0:
                    DQ[index] = np.nan
        
            # New feasable solution found
            Q = DQ
            blacklist = []
            
    DQ = np.nan_to_num(DQ)
    return DQ, np.sum(DQ * C)

In [85]:
C = np.array([
    [4, 5, 13],
    [12, 9, 11],
    [6, 14, 10],
    [30, 43, 30]
])
supply = [45, 25, 30, 15]
demand = [55, 25, 35]

In [90]:
MODI(C, supply, demand)

(array([[ 25.,  20.,   0.],
        [  0.,   5.,  20.],
        [ 30.,   0.,   0.],
        [  0.,   0.,  15.]]), 1095.0)

#### Usable links

- Solve Transportation Problem online: http://cbom.atozmath.com/CBOM/Transportation.aspx?q=modi
- MODI illustration: http://zoomin.idt.mdh.se/course/kpp227/Documents/MODI.pdf

# Traveling Salesman Problem
Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?

## Construction Heuristics
A heuristic is a technique designed for solving a problem more quickly when classic methods are too slow.

### Nearest Unvisited Neighbor Heuristic
The nearest neighbour (NN) algorithm (greedy algorithm) lets the salesman choose the nearest unvisited city as his next move. This algorithm quickly yields an effectively short route. For N cities randomly distributed on a plane, the algorithm on average yields a path 25% longer than the shortest possible path.

In [10]:
import numpy as np

def get_edges(route):
    return list(zip(route, route[1:]))

def length(D, route):
    edges = get_edges(route)
    return np.sum([D[e[0], e[1]] for e in edges])

In [11]:
def NN(D, start):
    route = [start]
    i = start
    while len(route) < len(D[0]):
        min_d = D[i, :].max() # initial value, must be non-zero
        min_j = D[i, :].argmax()
        for j, d in enumerate(D[i, :]):
            if j != i and j not in route and d < min_d:
                min_d = d
                min_j = j
        route.append(min_j)
        i = min_j
    route.append(start)
    return route, length(D, route)

In [12]:
def symmetrize(D):
    return D + D.T - np.diag(D.diagonal())

init_solution = [0, 2, 4, 1, 3, 5, 7, 6, 0]
D = np.array([
    [0, 4, 3, 4, 4, 7, 8, 8],
    [0, 0, 2, 6, 3, 6, 8, 7],
    [0, 0, 0, 4, 1, 4, 6, 5],
    [0, 0, 0, 0, 5, 4, 4, 5],
    [0, 0, 0, 0, 0, 3, 5, 4],
    [0, 0, 0, 0, 0, 0, 3, 2],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]
])
D = symmetrize(D)

In [13]:
NN(D, 0)

([0, 2, 4, 1, 3, 5, 7, 6, 0], 28)

### Insertion Heuristics
An intuitive approach to the TSP is to start with a subtour, i.e. a tour on small subsets of nodes, and then extend this tour by inserting the remaining nodes one after the other until all nodes have been inserted. The major difference between insertion schemes is the order in which the nodes are inserted.

In [14]:
def insertion(D):
    route = []
    for i in range(len(D)):
        if i == 0:
            route.append(0)
        elif i == 1:
            route.append(1)
            route.append(0)
        else:
            routes = []
            for j in range(1, len(route)-1):
                temp_route = route.copy()
                temp_route.insert(j, i)
                routes.append(temp_route)
            route = sorted(routes, key=lambda x: length(D, x))[0]
    return route, length(D, route)

In [15]:
insertion(D)

([0, 3, 6, 7, 5, 4, 2, 1, 0], 21)

## Improvement Heuristics

### The pairwise exchange or 2-opt heuristic
The pairwise exchange or 2-opt technique involves iteratively removing two edges and replacing these with two different edges that reconnect the fragments created by edge removal into a new and shorter tour. This is a special case of the k-opt method. Simple explained: The main idea behind it is to take a route that crosses over itself and reorder it so that it does not.
<img src="https://computingcore.files.wordpress.com/2013/04/kopt.png"/ width="600px">

#### Example

In [16]:
def _2opt(route, edges):
    e1, e2 = edges
    return route[:e1+1] + route[e1+1:e2+1][::-1] + route[e2+1:]

In [17]:
route = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'A']
_2opt(route, [0, 7])

['A', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']

## Chinese Postman Problem
All edges (not nodes) of a network have to be visited. Solving methods include transformation into an asymmetric TSP.

## Cost allocation: Shapley Value (game theory)
The solution, known as the Shapley value, has a nice interpretation in terms of expected marginal contribution. It is calculated by considering all the possible orders of arrival of the players into a game and giving each player his marginal contribution.

1. Calculate worth of 
    - individual players $\{A\},\{B\},\{C\}$
    - their coalitions $S\in\{AB\},\{AC\},\{BC\},\{ABC\}$
2. For all permutations, calculate the marginal contribution of each player
3. For each player, calculate the expected marginal contribution

| Permutation | A | B | C |
|-|-|-|-|
| ABC | v({A}) | v({AB}) - v({A}) | v({ABC}) - v({AB}) |
| ACB | v({A}) | v({ABC}) - v({AC}) | v({AC}) - v({A}) |
| BAC | v({AB}) - v({A}) | v({B}) | v({ABC}) - v({AB}) |
| BCA | v({ABC}) - v({BC}) | v({B}) | v({BC}) - v({B}) |
| CAB | v({AC}) - v({C}) | v({ABC}) - v({AC}) | v({C}) |
| CBA | v({ABC}) - v({BC}) | v({BC}) - v({C}) | v({C}) |

- Good example: http://faculty.econ.ucdavis.edu/faculty/bonanno/teaching/122/Shapley.pdf

In [82]:
import itertools
from collections import defaultdict
import pandas as pd

def shapley(D):
    players = list(range(1, len(D)))
    # Calculate worth of each player and coalition
    worth = {}
    for i in range(1, len(D)):
        for perm in itertools.permutations(players, i):
            # Worth doesn't depend on order within a coalition (= symmetry property)
            perm_set = tuple(set(perm))
            perm_len = length(D, [0] + list(perm) + [0])
            if perm_set not in worth:
                worth[perm_set] = perm_len
            elif perm_len < worth[perm_set]:
                # The worth of each coalition is the minimum distance covered within this coalition
                worth[perm_set] = perm_len

    # Calculate the expected marginal contribution of each player
    contributions = defaultdict(dict)
    for perm in itertools.permutations(players):
        for i, player in enumerate(perm):
            coalition = tuple(set(perm[:i + 1]))
            contributions[tuple(perm)][player] = worth[coalition]
            if i > 0:
                prev_coalition = tuple(set(perm[:i]))
                contributions[tuple(perm)][player] -= worth[prev_coalition]

    df = pd.DataFrame(list(contributions.values()), index=list(contributions.keys()))
    return df.mean().to_dict()

In [83]:
D = np.array([
    [0, 11, 18, 18],
    [8, 0, 7, 10],
    [18, 9, 0, 8],
    [20, 10, 11, 0]
])

In [84]:
shapley(D)

{1: 5.166666666666667, 2: 18.666666666666668, 3: 20.166666666666668}

# Vehicle Routing Problem
Extension of TSP by adding capacity constraints. Let there be number of demand points in a given area, each demanding an quantity of goods to be delivered to it. The goods in question are stored at a depot, where a fleet of vehicles is also stationed. Vehicles have identical maximum weight capacities and maximum routetime (or distance) constraints. They must all start and finish their routes at the depot. The problem is to obtain a set of delivery routes from the depot to the various demand points to minimize the total distance covered by the entire fleet.

## Heuristics

### 1. Route, 2. Cluster: TSP and revisit
Determine a tour without capacity constraints (TSP). Then go back to depot if the next customer cannot be visited with given capacity.

1. Optimal TSP solution
    - A => B => C => D => E => F => G => H => A
2. Clustering (length of each route can vary based on capacity constraints)
    - A => B => C => A
    - A => D => E => A
    - A => F => G => A
    - A => H => A

### 1. Cluster, 2. Route: Sweep method
Representation of customers in sequence of their polar coordinates.
<img src="http://images.slideplayer.com/15/4540330/slides/slide_5.jpg" width="500px">

In [20]:
import math

def sweep_method(XY, D, max_size):
    """
    XY: Matrix of customer locations on a map or grid
    D: distance matrix
    max_size: maximum cluster size
    """
    X, Y = zip(*XY)
    # Sort customers by angle between them and depot (counterclockwise)
    degrees = np.arctan2(Y[1:] - Y[0], X[1:] - X[0]) * 180 / np.pi
    degrees[degrees < 0] += 360
    
    clusters = []
    for i, _ in sorted(zip(range(1, len(XY)), degrees), key=lambda x: x[1]):
        if len(clusters) == 0:
            # First cluster
            clusters.append([0, i, 0])
        else:
            if len(clusters[-1]) == max_size + 2: # + 2x depot
                # Finish the cluster if constraints exceeeded
                clusters.append([0, i, 0])
            else:
                # Append new customer to the last cluster
                cluster = clusters[-1]
                cluster.insert(len(cluster) - 1, i)
                
                # Calculate the optimal TSP solution for the given cluster
                opt_edges = len(cluster) - 4, len(cluster) - 2
                opt_cluster = _2opt(cluster, opt_edges)
                if length(D, opt_cluster) < length(D, cluster):
                    clusters[-1] = opt_cluster
                    
    return [(cluster, length(D, cluster)) for cluster in clusters]

In [21]:
XY = np.array([
    [0, 0],
    [2, 0],
    [-1, 1],
    [1, 2],
    [-2, 2],
    [1, -2]
])
D = np.array([
    [0, 22, 36, 36, 50, 28],
    [22, 0, 14, 20, 58, 50],
    [36, 14, 0, 14, 63, 64],
    [36, 20, 14, 0, 51, 64],
    [50, 58, 63, 51, 0, 60],
    [28, 50, 64, 64, 60, 0]
])

In [22]:
sweep_method(XY, D, 3)

[([0, 1, 2, 3, 0], 86), ([0, 4, 5, 0], 138)]

### Route + Cluster simultaniously: Savings method
By far the best-known approach to the VRP problem is the "savings" algorithm of Clarke and Wright.
<img src="http://www.jtscm.co.za/index.php/jtscm/article/viewFile/90/89/322">

- Detailed description: http://ieor.berkeley.edu/~ieor151/lecture_notes/ieor151_lec18.pdf

In [23]:
import itertools

def savings_method(D, max_size):
    """
    D: distance matrix
    max_size: maximum cluster size
    """
    customers = list(range(1, len(D)))
    commutes = {}
    for c in customers:
        # Commute from depot to customer and back to depot
        commutes[c] = length(D, [0] + [c] + [0])
    
    perms = itertools.permutations(customers, 2)
    savings = []
    for perm in perms:
        c1, c2 = perm
        d1 = commutes[c1]
        d2 = commutes[c2]
        # Commute from depot to customer c1 to customer c2 and back to depot
        d = length(D, [0] + [c1, c2] + [0])
        # The distance we save by linking two customers together
        saving = d1 + d2 - d
        savings.append((perm, saving))
    ordered = sorted(savings, key=lambda x: x[1], reverse=True)

    clusters = []
    # Starting from the top of the list, build clusters of customers
    for c1, c2 in list(zip(*ordered))[0]:
        if len(clusters) == 0:
            # First cluster
            clusters.append([c1, c2])
        else:
            new_cluster = True
            for cluster in clusters:
                if c1 in cluster and c2 in cluster:
                    # If both already in cluster, ignore
                    new_cluster = False
                    break
                elif c1 in cluster or c2 in cluster:
                    # Proceed if the operation satisfies the size constraints
                    # Check for further constraints here
                    if len(cluster) < max_size:
                        # Customer can be added either on beginning or end
                        if c1 in cluster and c1 == cluster[-1]:
                            cluster.append(c2)
                        elif c2 in cluster and c2 == cluster[0]:
                            cluster.insert(0, c1)
                    new_cluster = False
                    break

            if new_cluster:
                # If there is no cluster with either c1 or c2, create new
                clusters.append([c1, c2])
    
    # Check whether all customers classified
    classified = set([c for cluster in clusters for c in cluster])
    for c in customers:
        if c not in classified:
            clusters.append([c])
    
    # Build routes by connecting customes to depot
    routes = [[0] + cluster + [0] for cluster in clusters]
    return [(route, length(D, route)) for route in routes]

In [24]:
D = np.array([
    [0, 20, 10, 14, 8, 16],
    [20, 0, 12, 22, 22, 21],
    [10, 12, 0, 10, 7, 19],
    [14, 22, 10, 0, 12, 29],
    [8, 22, 7, 12, 0, 16],
    [16, 21, 19, 29, 16, 0]
])

savings_method(D, 3)

[([0, 5, 1, 2, 0], 59), ([0, 3, 4, 0], 34)]

# Metaheuristics for TSP
Heuristics are often problem-dependent, that is, you define an heuristic for a given problem. Meta-heuristics are problem-independent techniques that can be applied to a broad range of problems.

Advantages:
- Finding fast solutions
- General solution procedures

Disadvantages:
- No guarantee of finding the optimum
- No measurement of solution quality

## Simulated Annealing
It is a metaheuristic to approximate global optimization in a large search space. It is often used when the search space is discrete (e.g., all tours that visit a given set of cities). For problems where finding an approximate global optimum is more important than finding a precise local optimum in a fixed amount of time, simulated annealing may be preferable to alternatives such as gradient descent.
<img src="https://image.slidesharecdn.com/part2letterassignment-150624063825-lva1-app6892/95/modelbased-user-interface-optimization-part-ii-letter-assignment-at-sicsa-summer-school-on-computational-interaction-2015-48-638.jpg?cb=1435128014" width="500px">

**Sometimes things really do have to get worse before they can get better.**

- Interactive example: http://toddwschneider.com/posts/traveling-salesman-with-simulated-annealing-r-and-shiny/
- Generic implementation: https://github.com/chncyhn/simulated-annealing-tsp/blob/master/anneal.py

In [25]:
def simulated_annealing(D, alpha, stop_it):
    """
    D: distance matrix
    alpha: temperature cooling rate
    stop_it: iteration to stop at
    """
    
    # Create the initial feasable solution. (e.g. NN)
    x, x_len = NN(D, 0)
    T = np.sqrt(x_len)
    X = []
    
    for i in range(stop_it):
        # Pick a new candidate tour at random from all neighbors of the existing tour.
        e1 = np.random.choice(range(len(x)-3)) # swap of minimum two nodes
        e2 = np.random.choice(range(e1, len(x)-1))
        opt_edges = [e1, e2]
        route = _2opt(x, opt_edges)
        l = length(D, route)
        delta = l - length(D, x)

        if delta <= 0:
            # If the candidate tour is better than the existing tour, accept it as the new tour
            x = route
            X.append(x)
        else:
            # If the candidate tour is worse than the existing tour, still maybe accept it, 
            # according to some probability. The probability of accepting an inferior tour 
            # is a function of how much longer the candidate is compared to the current tour, 
            # and the temperature of the annealing process. A higher temperature makes you 
            # more likely to accept an inferior tour.
            rand = np.random.random()
            p = np.exp(-delta/T)
            if rand < p:
                x = route
                X.append(x)
            
        # Cool down the temperature
        T *= alpha
    
    # Return the best solution found so far
    best = sorted(X, key=lambda x: length(D, x))[0]
    return best, length(D, best)

In [26]:
simulated_annealing(D, 0.995, 100)

([0, 2, 1, 5, 4, 3, 0], 85)

## Tabu Search
The objective for the Tabu Search algorithm is to constrain from returning to recently visited areas of the search space, referred to as cycling. 
<img src="https://image.slidesharecdn.com/part2letterassignment-150624063825-lva1-app6892/95/modelbased-user-interface-optimization-part-ii-letter-assignment-at-sicsa-summer-school-on-computational-interaction-2015-46-638.jpg?cb=1435128014" width="500px">

In [27]:
def tabu_search(D, nb_size, t_size, stop_it):
    """
    D: distance matrix
    nb_size: neighborhood size
    t_size: maximum Tabu list size
    stop_it: iteration to stop at
    """
    
    # Generate starting solution (e.g. using NN)
    x, _ = NN(D, 0)
    X = []
    T = []
    
    for i in range(stop_it):
        # At each step, look for the best solution in the neighborhood
        # A neighborhood to a given solution is defined as any other solution that 
        # is obtained by a pair wise exchange of any two nodes in the solution
        NB = []
        for j in range(nb_size):
            # Generate random opt2-operation
            e1 = np.random.choice(range(len(x)-3)) # swap of minimum two nodes
            e2 = np.random.choice(range(e1, len(x)-1))
            opt_edges = [e1, e2]
            route = _2opt(x, opt_edges)
            l = length(D, route)

            # To prevent the process from cycling in a small set of solutions, some attribute 
            # (pair of exchanged nodes) of recently visited solutions is stored in a Tabu List
            if opt_edges not in T:
                NB.append((opt_edges, route, l))
                
        # Choose the best neighbor
        opt_edges, route, l = sorted(NB, key=lambda n: n[2])[0]
        
        # Update only if new candidate better than the last candidate
        delta = l - length(D, x)
        if delta <= 0:
            x = route
            X.append(x)
            # Add the feature of the best candidate to the Tabu list
            T.append(opt_edges)
            # Pop first element of Tabu list if maximum size exceeded
            if len(T) > t_size:
                T = T[1:]
    
    # Return the best solution found so far
    best = sorted(X, key=lambda x: length(D, x))[0]
    return best, length(D, best)

In [28]:
tabu_search(D, 3, 3, 100)

([0, 4, 3, 2, 1, 5, 0], 79)

## Genetic Algorithm
Genetic algorithms are commonly used to generate high-quality solutions to optimization and search problems by relying on bio-inspired operators such as mutation, crossover and selection. 
<img src="https://media.nature.com/lw926/nature-assets/srep/2016/161121/srep37616/images_hires/srep37616-f3.jpg" width="500px">

- GA application on TSP: https://iccl.inf.tu-dresden.de/w/images/b/b7/GA_for_TSP.pdf
- Generic implementation: https://gist.github.com/josephmisiti/940cee03c97f031188ba7eac74d03a4f

In [29]:
def encode_tour(tour):
    # The mutation operator and the one-point crossover are likely to generate infeasible tours.
    # The TSP, as opposed to most problems tackled by genetic algorithms, is a pure ordering problem.
    # Namely, all chromosomes carry exactly the same values and differ only in the ordering of these values.
    # This implementation focuses on the relative order of the nodes to tacke this problem.
        
    tour = tour.copy()[:-1] # not include last (= first) value
    encoded = []
    # For each element in tour, order list starting from this element and save its index
    for i, node in enumerate(tour):
        encoded.append(sorted(tour[i:]).index(node))
    # Returns list of indices
    return encoded

def subtract(x, y):
    return [item for item in x if item not in y]

def decode_tour(encoded):
    ordered = range(len(encoded))
    decoded = []
    # For each index, find a non-repeating value corresponding to this index
    # from ordered list of all possible values
    for i, node in enumerate(encoded):
        remaining = subtract(range(len(encoded)), decoded)
        decoded.append(remaining[node])
    decoded.append(decoded[0]) # append last (= first) value
    return decoded

In [30]:
def random_population(n, m):
    # The process begins with a set of individuals which is called a Population. 
    # An individual is characterized by a set of parameters (variables) known as Genes. 
    # Genes are joined into a string to form a Chromosome (solution).
    
    # Generate random population with shape (N, M)
    P = []
    for i in range(n):
        ind = list(range(m - 1))
        np.random.shuffle(ind)
        ind.append(ind[0])
        P.append(ind)
    return P

def fitness(D, P):
    # The fitness of the chromosome is related to the tour length, which in turn 
    # depends on the ordering of the nodes. Since the TSP is a minimization problem, 
    # the tour lengths must be transformed so that high fitness values are associated 
    # with short tours. A well-known approach is to subtract each tour length to the 
    # maximum tour length found in the current population.
    
    values = np.array([length(D, p) for p in P])
    values = np.max(values) - values
    return sorted(zip(P, values), key=lambda x: x[1], reverse=True)

def weighted_select(D, P):
    # Two pairs of individuals (parents) are selected based on their fitness scores. 
    # The probability that an individual will be selected for reproduction is based 
    # on its fitness score. Individuals with high fitness have more chance to be 
    # selected for reproduction.
    
    F = fitness(D, P)
    _, fitness_values = zip(*F)
    fvsum = np.sum(fitness_values)
    weights = [fv / fvsum for fv in fitness_values]
    indices = np.random.choice(np.arange(0, len(P)), 2, p=weights, replace=False)
    return F[indices[0]][0], F[indices[1]][0]

def crossover(p1, p2):
    # For each pair of parents to be mated, a crossover point is chosen at random 
    # from within the genes. Offspring are created by exchanging the genes of parents 
    # among themselves until the crossover point is reached.
        
    # Generate two new individuals by a random crossover
    cut = np.random.choice(range(1, len(p1)))
    return p1[:cut] + p2[cut:], p2[:cut] + p1[cut:]

def mutate(ind, p):
    # In certain new offspring formed, some of their genes can be subjected to a 
    # mutation with a low random probability. Mutation occurs to maintain diversity 
    # within the population and prevent premature convergence.
        
    # Mutate each gene with certain probability
    mutated = []
    for i in range(len(ind)):
        if np.random.random() <= p:
            # Consider each gen as an index in encoded individual
            # The more to the right, the less an index can be
            allowed_len = len(ind)-len(mutated)
            mutated.append(np.random.choice(range(allowed_len)))
        else:
            mutated.append(ind[i])
    return mutated

In [31]:
def genetic_algorithm(D, n, p, stop_it):
    """
    D: distance matrix
    n: number of individuals in initial random population
    p: mutation probability
    stop_it: iteration to stop at
    """
    
    # Generate population
    P = random_population(n, len(D))
    
    for i in range(stop_it):
        # New generation
        
        # Select
        p1, p2 = weighted_select(D, P)

        # Encode
        p1 = encode_tour(p1)
        p2 = encode_tour(p2)
        
        # Crossover
        c1, c2 = crossover(p1, p2)
        
        # Mutate
        c1 = mutate(c1, p)
        c2 = mutate(c2, p)
            
        # Decode
        c1 = decode_tour(c1)
        c2 = decode_tour(c2)
        
        # Add to the Population
        P.append(c1)
        P.append(c2)
        
    # Return the best solution found so far
    best = sorted(P, key=lambda p: length(D, p))[0]
    return best, length(D, best)

In [32]:
genetic_algorithm(D, 10, 0.01, 100)

([0, 1, 2, 3, 4, 0], 62)

# Transportation Networks
Transportation networks generally refer to a set of links, nodes, and lines that represent the infrastructure or supply side of the transportation. The links have characteristics such as speed and capacity for roadways; frequency and travel time data are defined on transit links or lines for the transit system.

## Dynamic Optimization
Minimize costs of current decision and the follow-up costs. 

In [33]:
def dynamic_opt(T, H):
    """
    T: matrix of transportation costs (decision, node)
    S: matrix of switching costs (from decision, to decision)
    """
    stages = list(range(len(T[0])))
    decisions = list(range(len(H)))
    CM = np.zeros((len(decisions), len(stages)))
    DM = CM.copy()

    # Make decisions by going backwards in time
    for i, s in enumerate(stages[::-1]):
        for d1 in decisions:
            costs = []
            for d2 in decisions:
                # Costs by switching from one decision to another
                hcosts = S[d1, d2] if s > 0 else 0
                # Transportation costs associated with this decision
                tcosts = T[d2, s]
                # Follow-up costs associated with this decision
                fcosts = CM[d2, s + 1] if i > 0 else 0
                costs.append(hcosts + tcosts + fcosts)
            # Minimum costs by node and decision
            CM[d1, s] = min(costs)
            DM[d1, s] = np.array(costs).argmin()
    # Return matrix of costs and optimal decisions for each stage
    return CM, DM

In [34]:
T = np.array([
    [850, 1600, 800],
    [500, 1900, 500]
])
S = np.array([
    [0, 200],
    [200, 0]
])

In [35]:
dynamic_opt(T, S)

(array([[ 2900.,  2300.,   700.],
        [ 2900.,  2400.,   500.]]), array([[ 1.,  0.,  1.],
        [ 1.,  1.,  1.]]))

## Network Design

### Shortest Path: Dijkstra's Algorithm
Dijkstra's algorithm is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks.
<img src="http://www.baeldung.com/wp-content/uploads/2017/01/step8.png" width="400px">
- Explanation: https://www.youtube.com/watch?v=UG7VmPWkJmA

In [36]:
def successors(D, node):
    # Find successors of the current node
    return [s for s, d in enumerate(D[node]) if d != np.inf and s != node]

def shortest_path(D, source, target):
    # Find the shortest path from the source to the target
    dist = np.array([np.inf] * len(D))
    dist[source] = 0
    unvisited = list(range(len(D)))
    predecessors = {}
    
    while target in unvisited:
        # Choose an unvisited node dith the shortest distance to the source
        n1 = unvisited[dist[unvisited].argmin()]
        for n2 in successors(D, n1):
            if n2 in unvisited:
                # For all successors, update the distance to the source
                d = D[n1, n2]
                if dist[n1] + d < dist[n2]:
                    dist[n2] = dist[n1] + d
                    # Save the predecessor node
                    predecessors[n2] = n1
        unvisited.remove(n1)
    
    # Reconstruct the shortest path
    path = [target]
    while path[0] in predecessors:
        path.insert(0, predecessors[path[0]])
    return path, length(D, path)

In [37]:
D = np.array([
    [0, 7, 9, np.inf, np.inf, 14],
    [7, 0, 10, 15, np.inf, np.inf],
    [9, 10, 0, 11, np.inf, 2],
    [np.inf, 15, 11, 0, 6, np.inf],
    [np.inf, np.inf, np.inf, 6, 0, 9],
    [14, np.inf, 2, np.inf, 9, 0]
])

In [38]:
shortest_path(D, 0, 5)

([0, 2, 5], 11.0)

### Minimum Spanning Tree: Prim's Algorithm
Prim's algorithm is a greedy algorithm that finds a minimum spanning tree for a weighted undirected graph. This means it finds a subset of the edges that forms a tree that includes every vertex, where the total weight of all the edges in the tree is minimized.
<img src="https://he-s3.s3.amazonaws.com/media/uploads/146b47a.jpg" width="500px">

In [39]:
def MST(D):
    # Pick a random source node
    source = np.random.choice(range(len(D)))
    visited = [source]
    path = []
    
    while len(visited) < len(D):
        # Aggregate visited nodes as a big single node
        # Get all outgoing edges of this node
        crossing = []
        for n1 in visited:
            for n2 in successors(D, n1):
                # Successor should be an unknown node
                if n2 not in visited:
                    crossing.append((n1, n2))
        if len(crossing) == 0:
            break
        # Select the nearest successor
        min_edge = sorted(crossing, key=lambda x: D[x[0], x[1]])[0]
        path.append(min_edge)
        visited.append(min_edge[1])
    return path, sum(map(lambda x: length(D, x), path))

In [40]:
MST(D)

([(5, 2), (5, 4), (4, 3), (2, 0), (0, 1)], 33.0)

### Max Flow Problem: Ford-Fulkerson Algorithm
The idea behind the algorithm is as follows: as long as there is a path from the source (start node) to the sink (end node), with available capacity on all edges in the path, we send flow along one of the paths. Then we find another path, and so on.

- Detailed description: https://www.hackerearth.com/practice/algorithms/graphs/maximum-flow/tutorial/
- Video illustration: https://www.youtube.com/watch?v=Tl90tNtKvxs&t=181s
- Example: https://www.cs.princeton.edu/courses/archive/spring13/cos423/lectures/07DemoFordFulkerson.pdf

In [41]:
D = np.array([
    [0, 10, 10, np.inf, np.inf, np.inf],
    [np.inf, 0, 2, 4, 8, np.inf],
    [np.inf, np.inf, 0, np.inf, 9, np.inf],
    [np.inf, np.inf, np.inf, 0, np.inf, 10],
    [np.inf, np.inf, np.inf, 6, 0, 10],
    [np.inf, np.inf, np.inf, np.inf, np.inf, 0]
])

In [42]:
def max_flow(D, start, end):
    D = D.copy()
    max_flow = 0
    
    while True:
        # Augment the flow while there is path from source to sink
        path, path_len = shortest_path(D, start, end)
        if path_len == 0:
            break
            
        # Find minimum residual capacity of the edges along the path
        ordered = sorted([(e, length(D, e)) for e in get_edges(path)], key=lambda x: x[1])
        _, min_len = ordered[0]
        
        # Add path flow to overall flow
        max_flow += min_len
        
        # Update residual capacities of the edges and reverse edges along the path
        for (n1, n2), _ in ordered:
            D[n1, n2] -= min_len
            if D[n1, n2] == 0:
                D[n1, n2] = np.inf
            if D[n2, n1] == np.inf:
                D[n2, n1] = 0
            D[n2, n1] += min_len
    return D, max_flow

In [43]:
max_flow(D, 0, 5)

(array([[  0.,  inf,   1.,  inf,  inf,  inf],
        [ 10.,   0.,   2.,  inf,   2.,  inf],
        [  9.,  inf,   0.,  inf,  inf,  inf],
        [ inf,   4.,  inf,   0.,   5.,   1.],
        [ inf,   6.,   9.,   1.,   0.,  inf],
        [ inf,  inf,  inf,   9.,  10.,   0.]]), 19.0)

# Packaging Logistics
Packaging has a significant impact on the efficiency and effectiveness of retail supply chains, where improvements can be achieved through the adaptation and development of the concept of packaging logistics.

## Bin Packing
Matching objects with bins and reduce the number of bins.
<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0925527313001837-gr1.jpg" width="400px">

In [44]:
def bin_packing(capacity, sizes, priority):
    sizes = sizes.copy()
    bins = []
    while len(sizes) > 0:
        size = sizes[0]
        if len(bins) == 0 or sum(bins[-1]) + size > capacity:
            bins.append([])
            continue
        # Next fit: add to the last bin or open new bin
        if priority == 'next':
            bins[-1].append(size)
            sizes.remove(size)
        elif priority == 'best' or priority == 'worst':
            bins[-1].append(size)
            sizes.remove(size)
            while len(sizes) > 0:
                candidates = []
                for size2 in sizes:
                    if sum(bins[-1]) + size2 <= capacity:
                        candidates.append(size2)
                if len(candidates) > 0:
                    if priority == 'best':
                        # Best fit: add to a bin, so that remaining capacity is minimal
                        # Optimal with descending order
                        best = max(candidates)
                        bins[-1].append(best)
                        sizes.remove(best)
                    elif priority == 'worst':
                        # Worst fit: add to a bin, so that remaining capacity is maximal
                        worst = min(candidates)
                        bins[-1].append(worst)
                        sizes.remove(worst)
                else:
                    break
    return bins

In [45]:
bin_packing(12, [7, 6, 2, 4, 9, 5, 1, 7, 4, 2, 5, 8, 3, 4, 2], 'best')

[[7, 5], [6, 5, 1], [2, 9], [4, 8], [7, 4], [2, 4, 3, 2]]